In [1]:
import sys
# !{sys.executable} -m pip install pydash

In [2]:
import pandas as pd
from tqdm import tqdm
import numpy as np
import regex
import pydash
from sklearn.feature_extraction.text import CountVectorizer
from collections import defaultdict

tqdm.pandas()
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

/opt/conda/lib/python3.7/site-packages/tqdm/std.py:706: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [3]:
df = pd.read_csv('./french_tweets.csv')

In [4]:
df = df.append({'label': '0', 
                'text': "Je vais sapeurs pompiers, me dit qu'armée de l'air va rejoindre l'Armée de l'Air après les Sapeurs-Pompiers; je mange de l'air."}, 
               ignore_index=True
              )

In [5]:
df = pd.concat([df, df], axis=0, ignore_index=True)

In [6]:
df.shape

(3053450, 2)

In [7]:
forbidden_words = ["Gay", "Fuck", "Putain", "Fils de pute", "cul", "porno",
                   "putain de merde", "qu'il ne", "armée de l'air", "sapeurs-pompiers", "sapeurs pompiers"]

In [8]:
def preprocess_text(text):
    return regex.findall(r'(?u)\b[\w\w+]+\b', text)

In [9]:
def text_remove_dash(text):
    return regex.sub(r'-', ' ', text)

In [10]:
def preprocess_forbidden_words(words):
    forbidden_words = defaultdict(list)
    for word in words:
        forbidden_words[
            " ".join([subword for subword in preprocess_text(word.lower())])
        ].append(word.lower())
    return forbidden_words

In [11]:
def dataframe_process(dataframe, text_column, replace_by, forbidden_words):
    forbidden_words = preprocess_forbidden_words(forbidden_words)
    max_len_forbidden_words = max(map(lambda sent: len(sent.split()), forbidden_words.keys()))
    tqdm.pandas(desc="Preprocess text")
    dataframe['processed_text'] = dataframe[text_column].str.lower().progress_apply(
        lambda sent: " ".join([word for word in preprocess_text(sent)])
    )
    cv = CountVectorizer(
        strip_accents=None,
        encoding='latin-1',
        analyzer='word',
        token_pattern='\w+',
        lowercase=True,
        vocabulary=list(forbidden_words.keys()),
        stop_words=None,
        ngram_range=(1, max_len_forbidden_words)
    )
    transformed = cv.fit_transform(dataframe['processed_text'])
    vocab = np.array(cv.vocabulary)
    match_transformed = np.where(transformed.toarray() > 0)
    match_df = pd.DataFrame({
        'row_index': match_transformed[0],
        'cv_features': match_transformed[1]
    })
    final_match_df = match_df.groupby(
        'row_index'
    ).agg(list).merge(
        dataframe, left_on='row_index', right_index=True
    )
    del dataframe
    tqdm.pandas(desc="Search forbidden words")
    final_match_df['found_words'] = final_match_df['cv_features'].progress_apply(
        lambda features_indices: pydash.uniq(
            pydash.flatten([forbidden_words.get(w) for w in vocab[features_indices]])
        )
    )
    tqdm.pandas(desc="Prepare text anonymization patterns")
    final_match_df['replace_pattern'] = fr"\b" + final_match_df['found_words'].progress_apply(
        lambda words: pydash.uniq(
            pydash.flatten(words + [text_remove_dash(w) for w in words])
        )
    ).str.join(sep=fr"\b|\b") + fr"\b"
    tqdm.pandas(desc="Text anonymization")
    final_match_df['texte_remedié'] = final_match_df.progress_apply(
        lambda row: regex.sub(
            pattern=row['replace_pattern'], 
            repl=replace_by, 
            string=row[text_column], 
            flags=regex.IGNORECASE
        ),
        axis=1
    ).tolist()
    final_match_df['found_words'] = final_match_df['found_words'].str.join(sep=', ')
    del final_match_df['processed_text']
    del final_match_df['replace_pattern']
    del final_match_df['cv_features']
    return final_match_df

In [12]:
rez = dataframe_process(
    dataframe=df,
    text_column='text',
    replace_by='__XX__',
    forbidden_words=forbidden_words
)

/opt/conda/lib/python3.7/site-packages/tqdm/std.py:706: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel
Text anonymization: 100%|██████████| 18592/18592 [00:01<00:00, 15399.86it/s]


In [13]:
rez

,label,text,found_words,texte_remedié
row_index,,,,
1,0,"Est contrarié qu'il ne puisse pas mettre à jour son facebook en le télémaignant ... et peut-être pleurer en conséquence, l'école aujourd'hui aussi. blabla!",qu'il ne,"Est contrarié __XX__ puisse pas mettre à jour son facebook en le télémaignant ... et peut-être pleurer en conséquence, l'école aujourd'hui aussi. blabla!"
395,0,"S'est réveillé et a eu un accident - ""c'est pousser, il pousse!"" Il pleurait parce qu'il ne pouvait s'empêcher de mouiller son pantalon.",qu'il ne,"S'est réveillé et a eu un accident - ""c'est pousser, il pousse!"" Il pleurait parce __XX__ pouvait s'empêcher de mouiller son pantalon."
458,0,"Mon drain de baignoire est tiré: il existe 1 travail 2 fait, & amp; Il échoue. J'ai tout drano sur son cul, & amp; Il n'est pas encore drainant. Je veux me doucher, merci!",cul,"Mon drain de baignoire est tiré: il existe 1 travail 2 fait, & amp; Il échoue. J'ai tout drano sur son __XX__, & amp; Il n'est pas encore drainant. Je veux me doucher, merci!"
532,0,Fuck omg austins toujours là mais l'homme & lt; 3 love you,fuck,__XX__ omg austins toujours là mais l'homme & lt; 3 love you
613,0,"Je ne veux pas me réveiller tôt demain. Putain, travail!",putain,"Je ne veux pas me réveiller tôt demain. __XX__, travail!"
...,...,...,...,...
3052077,1,Je ne suis pas ce genre de garçon. mon bébé. Bien que je sois sûr qu'il ne serait pas heureux que je viens de dire bébé!,qu'il ne,Je ne suis pas ce genre de garçon. mon bébé. Bien que je sois sûr __XX__ serait pas heureux que je viens de dire bébé!
3052907,1,"L'accès sans fil a été installé dans l'observatoire de baldwin ... trop dommage qu'il ne soit jamais utilisé. Heureusement, cela peut aider à la couverture extérieure",qu'il ne,"L'accès sans fil a été installé dans l'observatoire de baldwin ... trop dommage __XX__ soit jamais utilisé. Heureusement, cela peut aider à la couverture extérieure"
3052913,1,"Vous le savez bien sûr, vous saurez quand je serai mieux. La bronchite et le stress frappe mon cul lol !!!!!",cul,"Vous le savez bien sûr, vous saurez quand je serai mieux. La bronchite et le stress frappe mon __XX__ lol !!!!!"
